In [ ]:
import cv2
import os
import numpy as np
import matplotlib as plt
import shutil, sys

In [ ]:
def add_random_straight_lines(image,height, width):
    
    num_lines = np.random.randint(1, 5) # number of lines to be added
    y0 = int(height/num_lines) # gap/width between each lines
    for i in range(num_lines):
        line_thickness = np.random.randint(1, 5)
        x1, x2 = 0, width # starting and ending x coordinates
        y = y0*(i+1) + np.random.randint(-0.05*height, 0.05*height) # y coordinate of line
        image = cv2.line(image, (x1, y), (x2, y), (0, 0, 0), thickness=line_thickness) #draw line
        prev_y = y
    return image
def add_random_text(image, height, width):
    
    closings = ['Sincerly', 'Regards', 'Yours truly', 'Best regards', 'Cordially']
    bottom_text = ['Amal Joseph', 'Steve Jobs', 'Larry Page', 'Paul Walker', 'Raja Ravi Varma', 'Katie Bouman', 'Ada Loveless']

    font = [cv2.FONT_HERSHEY_DUPLEX, cv2.FONT_HERSHEY_COMPLEX, cv2.FONT_HERSHEY_TRIPLEX, cv2.FONT_HERSHEY_COMPLEX_SMALL]
    y = np.random.randint(0.75*height, 1.02*height)
    x = np.random.randint(0.0005*width, 0.3*width)
    fontScale = np.random.random() + 0.7
    thickness = np.random.randint(1, 3)
    image = cv2.putText(image, np.random.choice(bottom_text), (x, y), np.random.choice(font), fontScale, (0, 0, 0), thickness, cv2.LINE_AA)
    return image
def process_image(image_path):    
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    image = add_random_text(image, height, width)
    image = add_random_straight_lines(image, height, width)
    return image


In [ ]:

image = cv2.imread(imagepath)
height, width, _ = image.shape
image = process_image(image, height, width)
plt.imshow(image)

In [ ]:

root_path = '/content/drive/MyDrive/gan_signdata_kaggle/images/'

for root, dirs, files in os.walk(root_path):
    for filename in files:
        shutil.move(os.path.join(root, filename), '/content/drive/MyDrive/gan_signdata_kaggle/A/')

In [ ]:
root_path = '/content/drive/MyDrive/gan_signdata_kaggle/A/'
for root, dirs, files in os.walk(root_path):
    for filename in files:
        image = process_image(os.path.join(root, filename))
        cv2.imwrite(f'/content/drive/MyDrive/gan_signdata_kaggle/B/{filename}', image)


In [ ]:
!cp -r '/content/drive/MyDrive/gan_signdata_kaggle/' '/content/drive/MyDrive/gan_signdata_kaggle (copy)/'

In [ ]:
# splitting into train and test
root = '/content/drive/MyDrive/gan_signdata_kaggle/'
srcA_path = '/content/drive/MyDrive/gan_signdata_kaggle/A/'
srcB_path = '/content/drive/MyDrive/gan_signdata_kaggle/B/'

trainA_path = '/content/drive/MyDrive/gan_signdata_kaggle (copy)/trainA/'
testA_path = '/content/drive/MyDrive/gan_signdata_kaggle (copy)/testA/'


trainB_path = '/content/drive/MyDrive/gan_signdata_kaggle (copy)/trainB/'
testB_path = '/content/drive/MyDrive/gan_signdata_kaggle (copy)/testB/'

In [ ]:
def split_data(src_path, train_path, test_path, split_ratio):
    files = np.array(os.listdir(src_path))
    np.random.shuffle(files)
    split_index = int(split_ratio * len(files))
    testA = files[0:split_index]
    trainA = files[split_index:]
    [shutil.move(os.path.join(src_path, path), os.path.join(train_path, path)) for path in trainA]
    [shutil.move(os.path.join(src_path, path), os.path.join(test_path, path)) for path in testA]

split_data(srcA_path, trainA_path, testA_path, 0.1)
split_data(srcB_path, trainB_path, testB_path, 0.1)

In [ ]:
os.rmdir(srcA_path)
os.rmdir(srcB_path)
os.rmdir('/content/drive/MyDrive/gan_signdata_kaggle/images/')

In [ ]:
!python train.py --dataroot ./datasets/gan_signdata_kaggle --name gan_signdata_kaggle --m

In [ ]:
from PIL import Image
import os, sys
size_images = dict()

for dirpath, _, filenames in os.walk(trainA_path):
    for path_image in filenames:
        image = os.path.abspath(os.path.join(dirpath, path_image))
        with Image.open(image) as img:
            width, heigth = img.size
            size_images[path_image] = {'width': width, 'heigth': heigth}
print(size_images)

{'06_028.png': {'width': 699, 'heigth': 314}, '01_0123026.PNG': {'width': 596, 'heigth': 312}, '04_0214022.PNG': {'width': 667, 'heigth': 355}, '04_024.png': {'width': 595, 'heigth': 382}, '04_0125022.PNG': {'width': 1043, 'heigth': 351}, '02_023.png': {'width': 819, 'heigth': 386}, '03_0101027.PNG': {'width': 680, 'heigth': 256}, '01_0205028.PNG': {'width': 696, 'heigth': 319}, '05_025.png': {'width': 295, 'heigth': 167}, '02_0212028.PNG': {'width': 868, 'heigth': 340}, '01_028.png': {'width': 734, 'heigth': 353}, '06_024.png': {'width': 659, 'heigth': 246}, '07_025.png': {'width': 321, 'heigth': 180}, '04_027.png': {'width': 616, 'heigth': 333}, '03_0119024.PNG': {'width': 505, 'heigth': 199}, '02_0125022.PNG': {'width': 975, 'heigth': 361}, '08_028.png': {'width': 825, 'heigth': 310}, '02_0212027.PNG': {'width': 642, 'heigth': 300}, '03_0214022.PNG': {'width': 831, 'heigth': 293}, '01_0126023.PNG': {'width': 623, 'heigth': 286}, '01_0101027.PNG': {'width': 641, 'heigth': 242}, '03_0

In [ ]:
import os
im_size = 512
def make_square(image, min_size=512, fill_color=(255, 255, 255, 0)):
    ''' Resize image as a square with signature in the center and black(transparent) strips at top and bottom. '''
    x, y = image.size
    size = max(min_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(image, (int((size - x) / 2), int((size - y) / 2)))
    new_im = new_im.resize((im_size, im_size))
    return new_im

def resize_images(path):
    ''' Function to resize the images to the ip format for gans. '''
    dirs = os.listdir(path)
    for item in dirs:
        if os.path.isfile(path+item):
            image = Image.open(path+item)
            image = make_square(image)
            image.save(path+item)

resize_images(trainA_path)
resize_images(trainB_path)
resize_images(testA_path)
resize_images(testB_path)